In [1]:
%load_ext autoreload
%autoreload 2
from topgun.optimiser.backtest import BacktestAnalytics
from topgun.reporting import Reporting
import numpy as np
import pandas as pd
import xlwings as xlw

In [2]:
wb = xlw.Book('BACKTEST.xlsm')

# index data from timeseries sheet
Assets = wb.sheets['TIMESERIES'].range('D1').options(pd.DataFrame, expand='table').value.iloc[3:,:].pct_change().dropna()
Assets.index = pd.to_datetime(Assets.index)

In [3]:
# Create a 60/40 portfolio
Assets['60/40']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*((0.6*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global']))

In [4]:
# Create the dataframe that holds the benchmark returns
bmktrtns = Assets[['60/40','STEFI']]
benchmark = '60/40'    # Becnhmark code from benchmarks

In [5]:
# Allocation to Carry Strategy
x1 = 0.02
x2 = 0.04
x3 = 0.06
x4 = 0.08
x5 = 0.10

rtns = pd.DataFrame() # create a empty data frame to store The Returns with allocation to Commodity Strategy

# Calculate the returns : Please note that Carry Strategy is financed from Equity

rtns['E2']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*(((0.6-x1)*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global'])+(x1*Assets['Minimum Drawdown Carry']))
rtns['E4']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*(((0.6-x2)*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global'])+(x2*Assets['Minimum Drawdown Carry']))
rtns['E6']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*(((0.6-x3)*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global'])+(x3*Assets['Minimum Drawdown Carry']))
rtns['E8']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*(((0.6-x4)*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global'])+(x4*Assets['Minimum Drawdown Carry']))
rtns['E10']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*(((0.6-x5)*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global'])+(x5*Assets['Minimum Drawdown Carry']))

In [6]:
bt = BacktestAnalytics(rtns, bmktrtns, bmks_as_rtns=True, benchmark=benchmark, Rf='STEFI')


In [7]:
bta = bt.run_backtest()

c:\users\b413863\appdata\local\continuum\anaconda3\lib\site-packages\statsmodels\regression\rolling.py:216: RuntimeWarning:

divide by zero encountered in log

c:\users\b413863\appdata\local\continuum\anaconda3\lib\site-packages\statsmodels\regression\rolling.py:216: RuntimeWarning:

divide by zero encountered in log



In [8]:
data = bt.backtest_summary()

In [9]:
data['summary']

,TR,Vol,Downside_Vol,Beta,Hitrate,Max_Drawdown,Max_XS_DD,Historic_VaR,Historic_CVaR,Sharpe,TE,IR,Calmar Ratio
Rf,0.071236,0.004928,NaN,-0.010093,1.000000,NaN,NaN,0.003418,0.002825,0.000000,0.084622,-0.623896,NaN
BMK,0.124031,0.083638,0.050655,1.000000,0.698980,-0.185102,0.000000,-0.029534,-0.039597,0.631233,0.000000,NaN,0.670068
E2,0.123828,0.083002,0.050133,0.992352,0.693878,-0.181175,-0.007814,-0.029252,-0.039239,0.633627,0.001004,-0.201927,0.683472
E4,0.123624,0.082373,0.049284,0.984703,0.698980,-0.177233,-0.015578,-0.028974,-0.038885,0.635988,0.002009,-0.202479,0.697523
E6,0.123419,0.081752,0.048589,0.977055,0.698980,-0.173276,-0.023292,-0.028699,-0.038535,0.638313,0.003013,-0.203030,0.712270
E8,0.123213,0.081138,0.047911,0.969406,0.698980,-0.169303,-0.030957,-0.028428,-0.038191,0.640599,0.004018,-0.203580,0.727765
E10,0.123006,0.080533,0.047542,0.961758,0.688776,-0.165315,-0.038571,-0.028161,-0.037850,0.642844,0.005022,-0.204130,0.744069


In [10]:
data['summary_appendix']

,RaR,Sortino,Avg Return,avg_win,avg_loss,Payoff,avg_drawdown,avg_XS_drawdown,avg_drawdown_days,avg_XS_drawdown_days,...,Treynor_Ratio,tail_ratio,xs_mean,xs_worst,xs_best,risk_of_ruin,profit_factor,recovery_factor,Omega_ratio,Modified_Sharpe_Ratio
Rf,14.456261,NaN,0.005752,0.005645,NaN,NaN,NaN,NaN,3.615385,47.75,...,-0.000000,2.302186,-0.004327,-0.097034,0.082315,0.000000e+00,inf,NaN,-inf,0.000000
BMK,1.482948,0.000000,0.010079,0.022354,-0.017334,1.289553,-0.015352,0.000000,3.615385,47.75,...,0.052795,1.555394,0.000000,0.000000,0.000000,4.874438e-148,2.322034,36.472655,2.931502,1.730742
E2,1.491869,-0.024494,0.010059,0.022247,-0.016859,1.319644,-0.015025,-0.003512,3.615385,47.75,...,0.052998,1.572173,-0.000020,-0.000651,0.001451,2.369085e-146,2.266667,37.153473,2.949117,1.744497
E4,1.500781,-0.050379,0.010039,0.022141,-0.016959,1.305569,-0.014718,-0.007011,3.615385,47.75,...,0.053202,1.581295,-0.000039,-0.001302,0.002903,4.874438e-148,2.322034,37.867423,2.966585,1.758273
E6,1.509677,-0.077505,0.010020,0.022035,-0.016780,1.313192,-0.014424,-0.010497,3.615385,47.75,...,0.053409,1.579237,-0.000059,-0.001952,0.004354,4.874438e-148,2.322034,38.616973,2.983728,1.772055
E8,1.518553,-0.105991,0.010000,0.021929,-0.016600,1.320979,-0.014130,-0.013971,3.615385,47.75,...,0.053618,1.580374,-0.000078,-0.002603,0.005805,4.874438e-148,2.322034,39.404837,3.001240,1.785830
E10,1.527401,-0.135059,0.009981,0.021822,-0.015888,1.373544,-0.013837,-0.017433,3.615385,47.75,...,0.053828,1.591280,-0.000098,-0.003254,0.007257,1.092348e-144,2.213115,40.234016,3.018488,1.799581


In [11]:
md=bt.big_bang(title="Backtest 60/40 Portfolio with allocation to Commodity Carry")

c:\users\b413863\appdata\local\continuum\anaconda3\lib\site-packages\statsmodels\regression\rolling.py:216: RuntimeWarning:

divide by zero encountered in log

c:\users\b413863\appdata\local\continuum\anaconda3\lib\site-packages\statsmodels\regression\rolling.py:216: RuntimeWarning:

divide by zero encountered in log

c:\users\b413863\appdata\local\continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning:

invalid value encountered in double_scalars



In [12]:
Reporting().md2html(md=md, title="Commodity Carry Strategy2")

'REPORT WRITTEN: Commodity Carry Strategy2'